<a href="https://colab.research.google.com/github/rabbitmetrics/openai-datascience/blob/main/notebooks/oai_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
  openai==1.3.7 \
  python-dotenv==1.0.0

## Load the API keys
Load the API keys from the env file. The env file is uploaded from your local machine to the Colab notebook by clicking the folder icon to the left and the clicking the upload icon.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## An OpenAI Function Call

This code snippet demonstrates how to use the OpenAI Python library to interact with the GPT-4 model. When making a request, include `response_format={ "type": "json_object" }` in the API call and also explicitely tell the model to return JSON.

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
customer = "Karen Bergstromm"

In [ ]:
# A simple prompt to extract information from "student_description" in a JSON format.

prompt = f'''
Give you best guess of age and gender based on the customer name and return it as a JSON object including the name:

This is the name to extract the information from:
{customer}
'''

In [ ]:
# Generate response with GPT-4

openai_response = client.chat.completions.create(
    model = 'gpt-4-1106-preview',
    messages = [{'role': 'user', 'content': prompt}],
    response_format={ "type": "json_object" }
)

In [ ]:
print(openai_response.choices[0].message.content)

{
  "name": "Karen Bergstromm",
  "age": "50-65",
  "gender": "Female"
}


# Dynamic Personas

In [ ]:
with open('personas.txt', 'r') as file:
    base_personas = file.read()
print(base_personas)

1. Young Professional Male: Age 25-35, interested in looking stylish yet comfortable for his daily commute and office environment. He values quality and brand reputation. His goal is to maintain a professional image while expressing his personal style.

2. Fitness Enthusiast Female: Age 20-30, she is a regular gym-goer and values athletic wear that is both functional and fashionable. Her goal is to stay fit and look good while doing it.

3. Trendy Teenager: Age 13-19, this persona is very interested in the latest trends and what their peers are wearing. They value affordability and the 'cool' factor. Their goal is to fit in and express their developing personal style.

4. Stylish Mum: Age 30-40, she is interested in looking fashionable while being able to comfortably manage her active kids. She values durability and versatility in clothing. Her goal is to maintain her sense of style while being a busy mum.

5. Sporty Kid: Age 6-12, interested in sports and outdoor activities. They valu

In [ ]:
pip install -qU klaviyo-api==5.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
from klaviyo_api import KlaviyoAPI

klaviyo_key=os.getenv('KLAVIYO_API_KEY')

klaviyo = KlaviyoAPI(klaviyo_key, max_delay=60, max_retries=3, test_host=None)

In [ ]:
john="01HK57639ZCVVAP57RZKZ0W0Z8"
karen="01HK4XAWEJE6DY8TCSMVK2G04E"

In [ ]:
# Function that extracts data for a customer from Klaviyo and returns as string that can be fed to LLM

def customer_data(custid):
    profile=klaviyo.Profiles.get_profile(
        custid
        )
    profile_string=str(profile['data']['attributes'])

    events=klaviyo.Events.get_events(fields_event=['event_properties'],
        filter=f"and(equals(profile_id,'{custid}'),equals(metric_id,'YtR57h'))",
        sort='-datetime'
        )
    purchases='Purchases: '
    for e in events['data']:
        purchases=purchases+e['attributes']['event_properties']['Name']+','
    return profile_string+', '+purchases

In [ ]:
# Get the data for a specific customer

data=customer_data(karen)

In [ ]:
data

"{'email': 'karen@rabbitpromotion.com', 'phone_number': None, 'external_id': None, 'anonymous_id': None, 'first_name': 'Karen', 'last_name': 'Bergstromm', 'organization': None, 'title': None, 'image': None, 'created': '2024-01-02T11:00:58+00:00', 'updated': '2024-01-02T16:45:43+00:00', 'last_event_date': '2024-01-02T16:45:43+00:00', 'location': {'address1': 'Lyngby Hovedgade', 'address2': None, 'city': 'Kongens Lyngby', 'country': 'Denmark', 'latitude': None, 'longitude': None, 'region': None, 'zip': '2800', 'timezone': 'Europe/Copenhagen', 'ip': None}, 'properties': {'Accepts Marketing': False, 'Shopify Tags': []}, 'subscriptions': {'email': {'marketing': {'consent': 'NEVER_SUBSCRIBED', 'timestamp': None, 'method': None, 'method_detail': None, 'custom_method_detail': None, 'double_optin': None, 'suppressions': [], 'list_suppressions': []}}, 'sms': None}}, Purchases: CONVERSE | TODDLER CHUCK TAYLOR ALL STAR AXEL MID,HERSCHEL | IONA,"

In [ ]:
persona_prompt = f'''
I have a customer described by the data here '{data}'.Here's a reference list of Persona categories {base_personas}

Give you best guess of age and gender based on the customer name.

Return the following :

Name
Persona category name
General interests
Behaviour
Age
Gender


Don't explain why just return it as a JSON object.

'''

In [ ]:
openai_response = client.chat.completions.create(
    model = 'gpt-4-1106-preview',
    messages = [{'role': 'user', 'content': persona_prompt}],
    response_format={ "type": "json_object" }
)

persona=openai_response.choices[0].message.content

print(persona)

{
  "Name": "Karen Bergstromm",
  "Persona category name": "Stylish Mum",
  "General interests": "Fashionable, comfortable clothing for managing active kids, durability, versatility",
  "Behaviour": "Values maintaining personal style while being a busy mum",
  "Age": "30-40",
  "Gender": "Female"
}


In [ ]:
# Update a profile with persona information

import json

persona=json.loads(persona)

payload={ "data": {
        "type": "profile",
        "id": karen,
        "attributes": {
            'properties':{
                'persona':persona['Persona category name'],
                'age':persona['Age'],
                'gender':persona['Gender'],
                'interest': persona['General interests'],
                'behavior': persona['Behaviour']
            }
                    }
            }
     }

klaviyo.Profiles.update_profile(karen, payload)

{'data': {'type': 'profile',
  'id': '01HK4XAWEJE6DY8TCSMVK2G04E',
  'attributes': {'email': 'karen@rabbitpromotion.com',
   'phone_number': None,
   'external_id': None,
   'anonymous_id': None,
   'first_name': 'Karen',
   'last_name': 'Bergstromm',
   'organization': None,
   'title': None,
   'image': None,
   'created': '2024-01-02T11:00:58+00:00',
   'updated': '2024-01-02T20:25:17.666226+00:00',
   'last_event_date': '2024-01-02T16:45:43+00:00',
   'location': {'address1': 'Lyngby Hovedgade',
    'address2': None,
    'city': 'Kongens Lyngby',
    'country': 'Denmark',
    'latitude': None,
    'longitude': None,
    'region': None,
    'zip': '2800',
    'timezone': 'Europe/Copenhagen',
    'ip': None},
   'properties': {'Accepts Marketing': False,
    'Shopify Tags': [],
    'persona': 'Stylish Mum',
    'age': '30-40',
    'gender': 'Female',
    'interest': 'Fashionable, comfortable clothing for managing active kids, durability, versatility',
    'behavior': 'Values maintain

In [ ]:
persona_name=persona['Persona category name']

system_prompt=f"""You are a world class marketing algorithm that write personalized emails for fashion brands.
                 I'm going to give you a Persona category from {base_personas}"""

user_prompt=f"""Write an email marketing campaign using the Attention-Interest-Desire-Action framework to grab
                the attention of the persona {persona_name} and persuade them to take action.

                 Start with a bold statement to get their attention, present information that piques their interest,
                 state the benefits of our product: 'Adidas Classic Sneakers' to create the desire: 'belonging'.

                 Use the angle 'Increase comfort'.

                 The first line should include a placeholder with first_name
                 in double curly brackets for the name of the customer."""

In [ ]:
email = client.chat.completions.create(
    model = 'gpt-4-1106-preview',
    messages = [{"role": "system", "content": system_prompt},
                {'role': 'user', 'content': user_prompt}]
)

print(email)

ChatCompletion(id='chatcmpl-8cfyIiDipkXfBgAR62Ilk38xjx52k', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Subject: Unleash Effortless Style & Comfort Every Day, {{first_name}}!\n\nHi {{first_name}},\n\nAre you ready to transform your busy mum-life with the perfect blend of style and comfort? Let's do it!\n\n**Chase after your little ones without skipping a beat!**\nWe understand that as a stylish mum, you're always on the move - playgrounds, school runs, and errands are just a part of your day. That's why we've crafted the ultimate solution for your active lifestyle - the Adidas Classic Sneakers. Imagine a sneaker that doesn't compromise on style or comfort, keeping you on-trend even on your busiest days.\n\n**Built for Mums, With Mums in Mind**\nOur Adidas Classic Sneakers are not just any ordinary footwear. With ultra-comfortable cushioning, these sneakers are designed to pamper your feet with every step. The durable design stands up to the rig

In [ ]:
print(email.choices[0].message.content)

Subject: Unleash Effortless Style & Comfort Every Day, {{first_name}}!

Hi {{first_name}},

Are you ready to transform your busy mum-life with the perfect blend of style and comfort? Let's do it!

**Chase after your little ones without skipping a beat!**
We understand that as a stylish mum, you're always on the move - playgrounds, school runs, and errands are just a part of your day. That's why we've crafted the ultimate solution for your active lifestyle - the Adidas Classic Sneakers. Imagine a sneaker that doesn't compromise on style or comfort, keeping you on-trend even on your busiest days.

**Built for Mums, With Mums in Mind**
Our Adidas Classic Sneakers are not just any ordinary footwear. With ultra-comfortable cushioning, these sneakers are designed to pamper your feet with every step. The durable design stands up to the rigors of motherhood, making each stride as pleasant as the first, no matter how many miles you've chalked up.

**Fashion That Feels Like Home**
Benefit from a

In [ ]:
email_content=email.choices[0].message.content

template = { "data": {
        "type": "template",
        "attributes": {
            "name": "Simple Template",
            "html": f"""
            <html>
                <body>
                    {email_content}
                </body>
            </html>
        """,
            "text": "hello this is a test template",
            "editor_type": "CODE"
        }
    } }

In [ ]:
klaviyo.Templates.create_template(template)

{'data': {'type': 'template',
  'id': 'RLKUnq',
  'attributes': {'name': 'Simple Template',
   'editor_type': 'CODE',
   'html': "<html>\n<head></head><body>\n                    Subject: Unleash Effortless Style &amp; Comfort Every Day, {{first_name}}!\n\nHi {{first_name}},\n\nAre you ready to transform your busy mum-life with the perfect blend of style and comfort? Let's do it!\n\n**Chase after your little ones without skipping a beat!**\nWe understand that as a stylish mum, you're always on the move - playgrounds, school runs, and errands are just a part of your day. That's why we've crafted the ultimate solution for your active lifestyle - the Adidas Classic Sneakers. Imagine a sneaker that doesn't compromise on style or comfort, keeping you on-trend even on your busiest days.\n\n**Built for Mums, With Mums in Mind**\nOur Adidas Classic Sneakers are not just any ordinary footwear. With ultra-comfortable cushioning, these sneakers are designed to pamper your feet with every step. Th

# Data Analysis with Tagging

In [ ]:
pip install -qU langchain==0.0.353

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd

campaign_frame = pd.read_json('campaign_performance.json')

In [ ]:
campaign_frame

,campaign_id,template_id,customer_id,firstname,lastname,email,customer_journey_stage,email_opened,email_clicked,email_conversion
0,2AQH77VVRXGWG8VSXOEOORZ8PQ,YivXH7,AshleyChavez,Ashley,Chavez,ashleychavez@rabbitpromotion.com,loyalty,0,0,0
1,W32D9L4DV0VNK2QXGUO62BVIY7,RLKUnq,AliciaSoto,Alicia,Soto,aliciasoto@rabbitpromotion.com,loyalty,1,0,0
2,IV6Q9BQMFD10M4OVU75BZB489L,RLKUnq,AnnaDeleon,Anna,Deleon,annadeleon@rabbitpromotion.com,welcome,1,1,1
3,SWG9EUT0SCB71F0F1DQFI8V74B,StAxfh,SandyGray,Sandy,Gray,sandygray@rabbitpromotion.com,retention,0,0,0
4,S5S9KWUO2T6QDOMBSGPNNBKY6G,YivXH7,PatriciaAnderson,Patricia,Anderson,patriciaanderson@rabbitpromotion.com,loyalty,1,0,0
...,...,...,...,...,...,...,...,...,...,...
4995,SWG9EUT0SCB71F0F1DQFI8V74B,StAxfh,JenniferWilliams,Jennifer,Williams,jenniferwilliams@rabbitpromotion.com,activation,0,0,0
4996,C7IZMEV17NOBKUXD6CZQMGIJSU,YivXH7,JoshuaPhillips,Joshua,Phillips,joshuaphillips@rabbitpromotion.com,retention,0,0,0
4997,VSP3EQ8HDI3SI0DUQG5JQ74LJY,StAxfh,RobinRobinson,Robin,Robinson,robinrobinson@rabbitpromotion.com,activation,1,0,0
4998,6PP9RUAMVFZWLVWM593UQUJ3L5,StAxfh,KimberlyMiller,Kimberly,Miller,kimberlymiller@rabbitpromotion.com,retention,0,0,0


In [ ]:
import requests, json

url = "https://a.klaviyo.com/api/templates/?sort=-created"

headers = {
    "accept": "application/json",
    "revision": "2023-12-15",
    "Authorization": f'Klaviyo-API-Key {klaviyo_key}'
}

response = requests.get(url, headers=headers)

templates=json.loads(response.text)['data']

In [ ]:
# Extract templates from the provided json file

with open("templates.json", "r") as read_file:
    templates = json.load(read_file)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain


schema = {
    "properties": {
        "background color": {"type": "string"},
        "font color": {"type": "string"},
        "font": {"type": "string"},
        "call to action": {"type": "string"},
        "fontsize": {"type": "integer"},
        "responsive yes no": {"type": "string"},
    }
}


chat = ChatOpenAI(temperature=0.2, model="gpt-4-1106-preview")
chain = create_tagging_chain(schema, chat)

In [ ]:
chain.run(templates[5]['attributes']['html'])

{'background color': '#f7f7f7',
 'font color': '#222222',
 'call to action': 'SHOP NOW',
 'fontsize': 14,
 'responsive yes no': 'yes'}

In [ ]:
angles=["Save time",
        'Avoid effort',
        'Escape mental or physical pain',
        'Increase comfort',
        'Improve health',
        'Improved status and luxury',
        'Feel loved',
        'Be praised/appreciated']

schema = {
    "properties": {

        "persuasiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how persuasive the sales pitch is (1-5), the higher the number the more persuasive",
        },
        "marketing_angle": {
            "type": "string",
            "enum": angles
        },
        "copywriting_framework": {
            "type": "string"
        },
    },
    "required": ["persuasiveness", "marketing_angle", "copywriting_framework"],
}


chain = create_tagging_chain(schema, chat)

In [ ]:
template_ids=[t['id'] for t in templates[0:4]]
plain_templates=[t for t in templates[0:4]]
features=[chain.run(t['attributes']['html']) for t in plain_templates]
template_dict=dict(zip(template_ids, features))

In [ ]:
template_dict

{'RLKUnq': {'persuasiveness': 4,
  'marketing_angle': 'Avoid effort',
  'copywriting_framework': 'AIDA'},
 'StAxfh': {'persuasiveness': 4,
  'marketing_angle': 'Increase comfort',
  'copywriting_framework': 'AIDA (Attention, Interest, Desire, Action)'},
 'YivXH7': {'persuasiveness': 4,
  'marketing_angle': 'Improved status and luxury',
  'copywriting_framework': 'AIDA'},
 'SgccCe': {'persuasiveness': 4,
  'marketing_angle': 'Improved status and luxury',
  'copywriting_framework': 'AIDA (Attention, Interest, Desire, Action)'}}

In [ ]:
campaign_frame['features']=campaign_frame.apply(lambda row: template_dict[row['template_id']],axis=1)
df_json = pd.json_normalize(campaign_frame['features'])
df_final = campaign_frame.join(df_json)

In [ ]:
# What is the converstion rate for each template
df_final.groupby('template_id')[['email_opened', 'email_clicked', 'email_conversion']].mean().reset_index()

In [ ]:
# What is the converstion rate for specific marketing angles
df_final.groupby('marketing_angle')[['email_opened', 'email_clicked', 'email_conversion']].mean().reset_index()

In [ ]:
# What is the converstion rate for specific marketing angles for different stages of the customer journey
df_final.groupby(['marketing_angle','customer_journey_stage'])[['email_opened', 'email_clicked', 'email_conversion']].mean().reset_index()